In [ ]:
import torch
import torchvision
print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)
print("CUDA is available:", torch.cuda.is_available())
import sys
!"{sys.executable}" -m pip install opencv-python matplotlib
!"{sys.executable}" -m pip install git+https://github.com/facebookresearch/segment-anything.git

!mkdir images
!curl -o images/truck.jpg https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
!curl -o images/groceries.jpg https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/groceries.jpg
!curl -O https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


In [ ]:
!pip install rasterio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install geopandas

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
import geopandas as gpd
from shapely.geometry import shape, box
from rasterio.features import shapes
import rasterio
from rasterio.mask import mask as rio_mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import math

# Paths to files
tif_file = "T34TCR_20230708T093549_TCI_10m.jp2"             # Original satellite image
mask_file = "ParcelaMaska.tif"        # Mask file with parcels
sam_checkpoint = "sam_vit_l_0b3195.pth"  # SAM model

#SAM parameters
SAM_tile_size = 512
SAM_overlap = 64
SAM_points_per_slide = 16
SAM_pred_iou_thresh = 0.9
SAM_stability_score_thresh = 0.9
output_file_name = "overlap6.shp"

# Load SAM model
model_type = "vit_l"
device = "cuda" if torch.cuda.is_available() else "cpu"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

# Load image
with rasterio.open(tif_file) as src_img:
    image = src_img.read()
    img_transform = src_img.transform
    img_crs = src_img.crs
    img_bounds = src_img.bounds
    img_width = src_img.width
    img_height = src_img.height

# Load mask
with rasterio.open(mask_file) as src_mask:
    mask = src_mask.read(1)  # Assuming mask is in the first band
    mask_transform = src_mask.transform
    mask_crs = src_mask.crs
    mask_bounds = src_mask.bounds

# Reproject mask if CRS do not match
if mask_crs != img_crs:
    print("CRS do not match. Reprojecting mask to image CRS...")
    transform, width, height = calculate_default_transform(
        src_mask.crs, img_crs, src_mask.width, src_mask.height, *src_mask.bounds)
    mask_reprojected = np.zeros((height, width), dtype=mask.dtype)
    reproject(
        source=mask,
        destination=mask_reprojected,
        src_transform=mask_transform,
        src_crs=mask_crs,
        dst_transform=transform,
        dst_crs=img_crs,
        resampling=Resampling.nearest
    )
    mask = mask_reprojected
    mask_transform = transform
else:
    print("CRS are the same.")

# Crop mask to image area
print("Cropping mask to image area...")
img_bbox = [box(*img_bounds)]
img_geo = gpd.GeoDataFrame({'geometry': img_bbox}, crs=img_crs)

with rasterio.open(mask_file) as src_mask:
    mask_cropped, mask_cropped_transform = rio_mask(
        src_mask, img_geo.geometry, crop=True, all_touched=True, nodata=0, filled=True
    )

mask = mask_cropped[0]  # Take the first (and only) layer
mask_transform = mask_cropped_transform

if (mask.shape[0] != img_height) or (mask.shape[1] != img_width):
    print("Resampling mask to match image dimensions...")
    mask_resampled = np.zeros((img_height, img_width), dtype=mask.dtype)
    reproject(
        source=mask,
        destination=mask_resampled,
        src_transform=mask_transform,
        src_crs=img_crs,
        dst_transform=img_transform,
        dst_crs=img_crs,
        resampling=Resampling.nearest
    )
    mask = mask_resampled
else:
    print("Mask and image dimensions are the same.")

# Ensure the mask is binary (0 and 1)
mask_binary = (mask > 0).astype(np.uint8)

# Prepare the image
image = np.transpose(image, (1, 2, 0))
if image.shape[2] > 3:
    image = image[:, :, :3]

if image.dtype != np.uint8:
    image = ((image - image.min()) / (image.max() - image.min()) * 255).astype(np.uint8)


# Define tile parameters
tile_size = SAM_tile_size  # Adjust as needed
overlap = SAM_overlap     # Adjust as needed

# Calculate the number of tiles
tiles_x = math.ceil((img_width - overlap) / (tile_size - overlap))
tiles_y = math.ceil((img_height - overlap) / (tile_size - overlap))

# Initialize final mask
final_mask = np.zeros((img_height, img_width), dtype=np.uint8)

# Process only the first few tiles
max_tiles_to_process = 40 # Set the maximum number of tiles you want to process

# Counter for processed tiles
tiles_processed = 0

# Process tiles
for i in range(tiles_y):
    for j in range(tiles_x):
        if tiles_processed >= max_tiles_to_process:
            break  # Exit the loop after processing the desired number of tiles
            
        # Calculate tile boundaries
        x_start = j * (tile_size - overlap)
        y_start = i * (tile_size - overlap)
        x_end = min(x_start + tile_size, img_width)
        y_end = min(y_start + tile_size, img_height)

        x_start = max(x_end - tile_size, 0)
        y_start = max(y_end - tile_size, 0)

        # Extract tile
        tile_image = image[y_start:y_end, x_start:x_end, :]
        tile_mask = mask_binary[y_start:y_end, x_start:x_end]

        # Skip tile if mask is empty
        if np.sum(tile_mask) == 0:
            continue
            
        # Apply mask to tile
        masked_tile_image = tile_image.copy()
        masked_tile_image[tile_mask == 0] = 0
        
        # Prepare tile for SAM
        tile_image_rgb = masked_tile_image
        if tile_image_rgb.dtype != np.uint8:
            tile_image_rgb = ((tile_image_rgb - tile_image_rgb.min()) / (tile_image_rgb.max() - tile_image_rgb.min()) * 255).astype(np.uint8)

        # Check if tile is not empty
        if np.sum(tile_image_rgb) == 0:
            continue

        # Run SAM on the tile
        mask_generator = SamAutomaticMaskGenerator(
            sam,
            points_per_side=SAM_points_per_slide,
            pred_iou_thresh=SAM_pred_iou_thresh,
            stability_score_thresh=SAM_stability_score_thresh,
        )

        print(f"Processing tile at position ({i}, {j})")
        tile_masks = mask_generator.generate(tile_image_rgb)

        # Combine masks from this tile
        tile_combined_mask = np.zeros((tile_image_rgb.shape[0], tile_image_rgb.shape[1]), dtype=np.uint8)
        for m in tile_masks:
            tile_combined_mask = np.maximum(tile_combined_mask, m['segmentation'].astype(np.uint8))

        # Place the tile mask into the final mask
        final_mask[y_start:y_end, x_start:x_end] = np.maximum(final_mask[y_start:y_end, x_start:x_end], tile_combined_mask)

        tiles_processed += 1

    if tiles_processed >= max_tiles_to_process:
        break  # Exit the outer loop as well

# Visualize the partial segmentation
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.imshow(final_mask, cmap='jet', alpha=0.5)
plt.axis('off')
plt.title("Partial Segmentation")
plt.show()

# Convert final_mask to polygons
geoms = []
for geom, value in shapes(final_mask, mask=None, transform=img_transform):
    if value == 1:
        geoms.append({'geometry': shape(geom)})

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(geoms, crs=img_crs)

# Save to Shapefile
output_shapefile = output_file_name
gdf.to_file(output_shapefile)

print(f"Partial segmentation results saved to {output_shapefile}")

In [ ]:
import rasterio
from rasterio.mask import mask as rio_mask
import geopandas as gpd
import numpy as np

# Putanje do fajlova
shp_file = "celo4.shp"  # shapefile sa parcelama segmentisanim SAM-om
crop_mask_file = "maskCrops.tif"  # Maska klasa useva

# Učitaj shapefile
gdf = gpd.read_file(shp_file)

# Učitaj masku useva
crop_data = rasterio.open(crop_mask_file)
crop_crs = crop_data.crs
crop_transform = crop_data.transform
crop_bounds = crop_data.bounds


# Proveri CRS, ako se ne poklapaju, reprojektuj gdf na crop_crs
if gdf.crs != crop_crs:
    gdf = gdf.to_crs(crop_crs)

major_classes = []

for idx, row in gdf.iterrows():
    geom = row['geometry']
    
    # Izreži crop masku prema poligonu
    # polygons argument mora biti [geom,]
    out_image, out_transform = rio_mask(crop_data, [geom], crop=True, nodata=255)
    # out_image je 3D [bands, height, width], ovde je verovatno [1, h, w]
    mask_array = out_image[0]  # pretpostavka da je samo jedan band
    
    # Izbaci vrednosti koje su nodata (ako ih ima)
    # Pod pretpostavkom da je nodata 255 ili nesto slicno
    # Pogledaj crop mask file i vidi koji je nodata
    # Ako nodata nije definisana, a imas 20 ili 6 klasa, mozda nema nodata
    # Ako ima nevalidnih vrednosti (npr. >20), izbaci ih
    valid_pixels = mask_array[(mask_array >=0) & (mask_array <= 20)]  # pretpostavka da su klase 0 do 20

    if len(valid_pixels) == 0:
        # Ako nema validnih piksela, mozda stavi major_class=20 (neklasifikovano) ili ostavi np.nan
        major_class = 20  
    else:
        # Nadji najcescu vrednost
        vals, counts = np.unique(valid_pixels, return_counts=True)
        major_class = vals[np.argmax(counts)]
    
    major_classes.append(major_class)

# Dodaj kolonu u gdf
gdf['major_class'] = major_classes

# Snimi ažurirani shapefile
gdf.to_file("celo4_with_class.shp")
print("Added major_class to each polygon")